In [1]:
#!pip install flask==0.12.2

In [2]:
from keras.preprocessing.image import img_to_array
from keras.applications import imagenet_utils
from PIL import Image
import numpy as np
import flask
import io
from keras.models import *
from keras.models import model_from_yaml
import string

/home/ubuntu/anaconda/envs/py36-venv/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [3]:
# initialize our Flask application and the Keras model
app = flask.Flask(__name__)
model = None

## Load the pre-trained model weights and architecture

In [4]:
def load_model():
    global model

    with open('models/captcha_model_structure.yaml', 'r') as myfile:
        yaml_string=myfile.read()
        
    model = model_from_yaml(yaml_string)
    model.load_weights('models/captcha_model_weight.h5')

In [5]:
def prepare_image(image, target):
    # if the image mode is not RGB, convert it
    if image.mode != "RGB":
        image = image.convert("RGB")

    # resize the input image and preprocess it
    image = image.resize(target)
    image = img_to_array(image)
    image = np.expand_dims(image, axis=0)

    return image

def decode(y):
    characters = string.digits + string.ascii_uppercase
    y = np.argmax(np.array(y), axis=2)[:,0]
    return ''.join([characters[x] for x in y])

## Predict captcha by uploaded image

In [6]:
@app.route("/predict", methods=["POST"])
def predict():

    # ensure an image was properly uploaded to our endpoint
    if flask.request.method == "POST":
        if flask.request.files.get("image"):
            # read the image in PIL format
            image = flask.request.files["image"].read()
            image = Image.open(io.BytesIO(image))

            # preprocess the image for classification model
            image = prepare_image(image, target=(170, 80))
            # predict text from captcha image
            pred = model.predict(image)
            result = decode(pred)

    return flask.jsonify(result)

In [ ]:
load_model()
app.run()

 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [05/Aug/2018 23:49:31] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [05/Aug/2018 23:49:53] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [05/Aug/2018 23:49:58] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [05/Aug/2018 23:50:09] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [05/Aug/2018 23:50:16] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [05/Aug/2018 23:50:21] "POST /predict HTTP/1.1" 200 -


## UnsupportedOperation: not writable when calling app.run()

This is a bug for the latest Flask in python 3.6.  

vi ~/anaconda/envs/py36-venv/lib/python3.6/site-packages/click/utils.py

Change file=None to below

def echo(message=None, file=sys.stdout, nl=True, err=False, color=None):